In [1]:
import pandas as pd
import numpy as np
from scipy.stats import lognorm
import matplotlib.pyplot as plt
import shared

ve_or_ju = "ve"
year = 2019

In [2]:
running_order = pd.read_csv(f"data/running_order_2019_with_estimates_{ve_or_ju}.tsv", delimiter="\t")
display(running_order.shape)
running_order.head()

(6759, 20)

,Unnamed: 0,team_id,team,team_base_name,team_country,leg,leg_dist,name,leg_nro,orig_name,num_runs,pred_log_mean,pred_log_std,predicted,log_q_low,log_q_high,log_std,log_std_fixed,final_pace_mean,final_pace_std
0,0,1,Stora Tuna OK 1,Stora Tuna OK,SWE,1,6.0,tilda östberg,1,Tilda Östberg,1.0,1.8857,0.0000,6.511986,1.881337,1.978520,0.048591,0.048591,1.8857,0.067659
1,1,1,Stora Tuna OK 1,Stora Tuna OK,SWE,2,5.7,magdalena olsson,2,Magdalena Olsson,6.0,1.9556,0.0886,6.992522,1.765439,1.961637,0.098099,0.098099,1.9556,0.088600
2,2,1,Stora Tuna OK 1,Stora Tuna OK,SWE,3,7.3,julia jakob,3,Julia Jakob,1.0,1.7824,0.0000,6.569075,1.798748,2.018901,0.110076,0.110076,1.7824,0.110076
3,3,1,Stora Tuna OK 1,Stora Tuna OK,SWE,4,7.9,tove alexandersson,4,Tove Alexandersson,6.0,1.8027,0.0586,6.290935,1.791712,1.973971,0.091129,0.091129,1.8027,0.067659
4,4,2,Göteborg Majorna OK 1,Göteborg Majorna OK,SWE,1,6.0,amanda hansson,1,Amanda Hansson,2.0,2.0053,0.0358,7.531656,1.880709,2.035837,0.077564,0.077564,2.0053,0.077564


In [3]:
order = running_order[["team_id", "team", "leg", "leg_dist", "orig_name", "final_pace_mean", "final_pace_std"]]


In [4]:
order = order.rename(index=str, columns={"orig_name": "name", "leg_dist": "dist", "final_pace_mean": "log_mean", "final_pace_std": "log_std"})

order.head()



,team_id,team,leg,dist,name,log_mean,log_std
0,1,Stora Tuna OK 1,1,6.0,Tilda Östberg,1.8857,0.067659
1,1,Stora Tuna OK 1,2,5.7,Magdalena Olsson,1.9556,0.088600
2,1,Stora Tuna OK 1,3,7.3,Julia Jakob,1.7824,0.110076
3,1,Stora Tuna OK 1,4,7.9,Tove Alexandersson,1.8027,0.067659
4,2,Göteborg Majorna OK 1,1,6.0,Amanda Hansson,2.0053,0.077564


In [5]:
by_teams = order.set_index(["team_id", "leg"]).unstack()
by_teams.head()

team                                                \
leg                          1                      2                      3   
team_id                                                                        
1              Stora Tuna OK 1        Stora Tuna OK 1        Stora Tuna OK 1   
2        Göteborg Majorna OK 1  Göteborg Majorna OK 1  Göteborg Majorna OK 1   
3                   MS Parma 1             MS Parma 1             MS Parma 1   
4          Järla Orientering 1    Järla Orientering 1    Järla Orientering 1   
5               Alfta ÖSA OK 1         Alfta ÖSA OK 1         Alfta ÖSA OK 1   

                               dist                               name  \
leg                          4    1    2    3    4                   1   
team_id                                                                  
1              Stora Tuna OK 1  6.0  5.7  7.3  7.9       Tilda Östberg   
2        Göteborg Majorna OK 1  6.0  5.7  7.3  7.9      Amanda Hansson   
3                   MS Parma 1  6.0  5.7  7.3  7.9   Anastasia Rudnaya   
4          Järla Orientering 1  6.0  5.7  7.3  7.9       Sara Sjökvist   
5               Alfta ÖSA OK 1  6.0  5.7  7.3  7.9  Galina Vinogradova   

                                                                       \
leg                        2                    3                   4   
team_id                                                                 
1           Magdalena Olsson          Julia Jakob  Tove Alexandersson   
2              Runa Fremstad     Kristine Berglia          Lisa Holer   
3             Anni Heikkonen       Tiia Istolahti       Maija Sianoja   
4        Anna Dahlgren-Rosén  Elin Hemmyr-Skantze      Sofie Bachmann   
5            Josefine Heikka         Irina Nyberg    Natalia Gemperle   

        log_mean                            log_std                      \
leg            1        2       3       4         1         2         3   
team_id                                                                   
1         1.8857  1.95560  1.7824  1.8027  0.067659  0.088600  0.110076   
2         2.0053  2.00810  1.9581  1.9854  0.077564  0.087900  0.067659   
3         1.8899  1.94010  1.8669  1.9157  0.067659  0.108600  0.128204   
4         1.9748  1.94591  1.8086  1.9289  0.112800  0.182322  0.124320   
5         1.8949  1.88830  1.9000  1.8720  0.110100  0.117383  0.095300   

                   
leg             4  
team_id            
1        0.067659  
2        0.097276  
3        0.067659  
4        0.087344  
5        0.125945

In [6]:
display(by_teams.shape)
# remove teams that have not nominated runners for all legs
by_teams = by_teams[np.all(np.isfinite(by_teams["log_mean"]), axis=1)]
display(by_teams.shape)

(1697, 20)

(1682, 20)

In [7]:
num_legs = shared.num_legs[ve_or_ju]
changeover_closing_timedelta = shared.changeover_closing[ve_or_ju][year] - shared.start_timestamp[ve_or_ju][year]
changeover_closing_mins = changeover_closing_timedelta.total_seconds() / 60
display(changeover_closing_mins)

270.0

In [8]:
if ve_or_ju == "ju":
    dark_period_start_timedelta = shared.dark_period[year]["start"] - shared.start_timestamp[ve_or_ju][year]
    dark_period_start_mins = dark_period_start_timedelta.total_seconds() / 60
    dark_period_end_timedelta = shared.dark_period[year]["end"] - shared.start_timestamp[ve_or_ju][year]
    dark_period_end_mins = dark_period_end_timedelta.total_seconds() / 60
else:
    # hack to disbale calculation for Venlas
    dark_period_start_mins = 10000
    dark_period_end_mins = -1
    

In [9]:

# Sum of log normal variables is not defined 
# so we simulate 10000 runs for each user and sum them and then do statistics on simulated results 
def simulate_relay_estimates(row):
    samples = pd.DataFrame()
    for i in range(1, num_legs + 1):
        if np.isnan(row["log_mean"][i]):
            print(row["log_mean"])
            print(row["name"])
        samples[i] = row["dist"][i] * lognorm.rvs(s = row["log_std"][i], scale = np.exp(row["log_mean"][i]), size = 10000)

    personal_start95 = samples.quantile(0.025)
    personal_end95 = samples.quantile(0.975)
    personal_median = samples.median()
        
    samples_sums = pd.DataFrame()
    # leg_1 
    # leg_1 + leg_2
    # leg_1 + leg_2 + leg_3
    # ...
    for i in range(1,num_legs + 1):
        samples_sums[i] = np.sum([ samples[j] for j in range(1,i+1) ], axis=0)

    start95 = samples_sums.quantile(0.025)
    end95 = samples_sums.quantile(0.975)
    medians = samples_sums.median()
    means = samples_sums.mean()
    
    before_changeover_closing_samples = samples_sums < changeover_closing_mins
    
    mass_start = 1 - before_changeover_closing_samples.mean()
    # mass start of runner depends on arrival of previous runner
    mass_start = mass_start.shift(1, fill_value=0)

    not_dark_during_leg_samples = pd.DataFrame()
    for i in range(1,num_legs + 1):
        if i == 1:
            start = 0
        else:    
            start = samples_sums[i - 1]
            
        finish = samples_sums[i]
        finish_before_sunset = finish < dark_period_start_mins
        start_after_sunrise = start > dark_period_end_mins
        not_dark_during_leg_samples[i] = finish_before_sunset | start_after_sunrise 
    
    dark_during_leg = 1 - not_dark_during_leg_samples.mean()    
    
    sum_logs = np.log(samples_sums)
    sum_log_means = np.mean(sum_logs)
    sum_log_stds = np.std(sum_logs)
    
    """
    for i in range(1, num_legs + 1):
        bins = int(samples_sums[i].max() - samples_sums[i].min())
        name = row["name"][i]
        plt.title(f"{name} bins = {bins}")
        plt.hist(samples_sums[i], bins=bins)
        #plt.axvline(x=row["fin_real"][i], color="r")        
        plt.axvline(x=medians[i], color="g")
        plt.axvline(x=means[i], color="yellow")
        plt.show()    
    """
    
    """
    bins = int( (samples_sums.max().max() - samples_sums.min().min()) / 5) 
    plt.figure(figsize=(20, 6))
    plt.title(f"Whole team, bins = {bins}")
    plt.hist([samples_sums[1], samples_sums[2], samples_sums[3], samples_sums[4], samples_sums[5], samples_sums[6], samples_sums[7]], bins=bins)
    for i in range(1, num_legs + 1):
        #plt.axvline(x=row["fin_real"][i], color="r")
        plt.axvline(x=medians[i], color="g")

    plt.show()
    """
    
    fin_start95_dict = {f"fin_start95_{leg}" : start95.values[leg-1] for leg in range(1, num_legs + 1)}
    fin_end95_dict = {f"fin_end95_{leg}" : end95.values[leg-1] for leg in range(1, num_legs + 1)}
    fin_median_dict = {f"fin_median_{leg}" : medians.values[leg-1] for leg in range(1, num_legs + 1)}
    fin_mean_dict = {f"fin_mean_{leg}" : means.values[leg-1] for leg in range(1, num_legs + 1)}
    fin_sum_log_means_dict = {f"fin_sum_log_mean_{leg}" : sum_log_means.values[leg-1] for leg in range(1, num_legs + 1)}
    fin_sum_log_stds_dict = {f"fin_sum_log_std_{leg}" : sum_log_stds.values[leg-1] for leg in range(1, num_legs + 1)}
    mass_start_dict = {f"mass_start_{leg}" : mass_start.values[leg-1] for leg in range(1, num_legs + 1)}
    dark_during_leg_dict = {f"dark_during_leg_{leg}" : dark_during_leg.values[leg-1] for leg in range(1, num_legs + 1)}

    personal_start95_dict = {}
    personal_end95_dict = {}
    personal_median_dict = {}
    for leg in range(1, num_legs + 1):
        personal_start95_dict[f"personal_start95_{leg}"] = personal_start95.values[leg-1]
        personal_end95_dict[f"personal_end95_{leg}"] = personal_end95.values[leg-1]
        personal_median_dict[f"personal_median_{leg}"] = personal_median.values[leg-1]
        
    new_cols = {**fin_start95_dict, 
                **fin_end95_dict, 
                **fin_median_dict, 
                **fin_mean_dict, 
                **fin_sum_log_means_dict, 
                **fin_sum_log_stds_dict, 
                **mass_start_dict, 
                **dark_during_leg_dict,
                **personal_start95_dict,
                **personal_end95_dict,
                **personal_median_dict}

    #print(start95.values)
    #print(new_cols)
    return pd.Series(new_cols)

relay_estimates = by_teams.apply(simulate_relay_estimates, axis=1)
relay_estimates.head()

,fin_start95_1,fin_start95_2,fin_start95_3,fin_start95_4,fin_end95_1,fin_end95_2,fin_end95_3,fin_end95_4,fin_median_1,fin_median_2,...,personal_start95_3,personal_start95_4,personal_end95_1,personal_end95_2,personal_end95_3,personal_end95_4,personal_median_1,personal_median_2,personal_median_3,personal_median_4
team_id,,,,,,,,,,,,,,,,,,,,,
1,34.575157,71.707841,111.591144,158.221538,45.098622,89.106250,136.843093,186.331715,39.566249,79.937558,...,35.116472,42.004915,45.098622,47.858028,53.721062,54.841958,39.566249,40.252988,43.444549,47.911052
2,38.313828,77.769334,127.402399,181.381610,51.842862,97.337735,151.443076,213.782269,44.577523,87.242081,...,45.309450,47.738844,51.842862,50.313313,58.881223,69.746518,44.577523,42.452216,51.688998,57.565351
3,34.758806,70.358247,112.911125,164.810583,45.383861,90.374947,143.650069,198.952830,39.717437,79.549918,...,36.813392,46.970873,45.383861,48.863960,60.771374,61.264849,39.717437,39.692932,47.353545,53.649774
4,34.794574,68.221485,109.429456,161.947717,53.844799,103.247493,151.188169,207.973097,43.269818,83.426493,...,34.915468,45.647306,53.844799,56.826576,57.047350,64.585792,43.269818,39.875924,44.651193,54.312177
5,32.026728,66.211810,111.957168,159.454191,49.411586,91.319313,143.450301,199.589341,39.877024,77.756903,...,40.335852,40.079235,49.411586,47.488400,58.636488,65.399162,39.877024,37.609373,48.730086,51.195256


In [10]:
# Flatten the troublesome multi-index to field_{leg} etc...
by_teams_flat = by_teams.copy()
by_teams_flat.columns = [f'{x[0]}_{x[1]}' for x in by_teams_flat.columns]
by_teams_flat.reset_index()
by_teams_flat.head().round(2)

,team_1,team_2,team_3,team_4,dist_1,dist_2,dist_3,dist_4,name_1,name_2,name_3,name_4,log_mean_1,log_mean_2,log_mean_3,log_mean_4,log_std_1,log_std_2,log_std_3,log_std_4
team_id,,,,,,,,,,,,,,,,,,,,
1,Stora Tuna OK 1,Stora Tuna OK 1,Stora Tuna OK 1,Stora Tuna OK 1,6.0,5.7,7.3,7.9,Tilda Östberg,Magdalena Olsson,Julia Jakob,Tove Alexandersson,1.89,1.96,1.78,1.80,0.07,0.09,0.11,0.07
2,Göteborg Majorna OK 1,Göteborg Majorna OK 1,Göteborg Majorna OK 1,Göteborg Majorna OK 1,6.0,5.7,7.3,7.9,Amanda Hansson,Runa Fremstad,Kristine Berglia,Lisa Holer,2.01,2.01,1.96,1.99,0.08,0.09,0.07,0.10
3,MS Parma 1,MS Parma 1,MS Parma 1,MS Parma 1,6.0,5.7,7.3,7.9,Anastasia Rudnaya,Anni Heikkonen,Tiia Istolahti,Maija Sianoja,1.89,1.94,1.87,1.92,0.07,0.11,0.13,0.07
4,Järla Orientering 1,Järla Orientering 1,Järla Orientering 1,Järla Orientering 1,6.0,5.7,7.3,7.9,Sara Sjökvist,Anna Dahlgren-Rosén,Elin Hemmyr-Skantze,Sofie Bachmann,1.97,1.95,1.81,1.93,0.11,0.18,0.12,0.09
5,Alfta ÖSA OK 1,Alfta ÖSA OK 1,Alfta ÖSA OK 1,Alfta ÖSA OK 1,6.0,5.7,7.3,7.9,Galina Vinogradova,Josefine Heikka,Irina Nyberg,Natalia Gemperle,1.89,1.89,1.90,1.87,0.11,0.12,0.10,0.13


In [11]:
estimates = pd.concat([by_teams_flat, relay_estimates], axis=1, join='inner')

In [12]:
estimates.head()

,team_1,team_2,team_3,team_4,dist_1,dist_2,dist_3,dist_4,name_1,name_2,...,personal_start95_3,personal_start95_4,personal_end95_1,personal_end95_2,personal_end95_3,personal_end95_4,personal_median_1,personal_median_2,personal_median_3,personal_median_4
team_id,,,,,,,,,,,,,,,,,,,,,
1,Stora Tuna OK 1,Stora Tuna OK 1,Stora Tuna OK 1,Stora Tuna OK 1,6.0,5.7,7.3,7.9,Tilda Östberg,Magdalena Olsson,...,35.116472,42.004915,45.098622,47.858028,53.721062,54.841958,39.566249,40.252988,43.444549,47.911052
2,Göteborg Majorna OK 1,Göteborg Majorna OK 1,Göteborg Majorna OK 1,Göteborg Majorna OK 1,6.0,5.7,7.3,7.9,Amanda Hansson,Runa Fremstad,...,45.309450,47.738844,51.842862,50.313313,58.881223,69.746518,44.577523,42.452216,51.688998,57.565351
3,MS Parma 1,MS Parma 1,MS Parma 1,MS Parma 1,6.0,5.7,7.3,7.9,Anastasia Rudnaya,Anni Heikkonen,...,36.813392,46.970873,45.383861,48.863960,60.771374,61.264849,39.717437,39.692932,47.353545,53.649774
4,Järla Orientering 1,Järla Orientering 1,Järla Orientering 1,Järla Orientering 1,6.0,5.7,7.3,7.9,Sara Sjökvist,Anna Dahlgren-Rosén,...,34.915468,45.647306,53.844799,56.826576,57.047350,64.585792,43.269818,39.875924,44.651193,54.312177
5,Alfta ÖSA OK 1,Alfta ÖSA OK 1,Alfta ÖSA OK 1,Alfta ÖSA OK 1,6.0,5.7,7.3,7.9,Galina Vinogradova,Josefine Heikka,...,40.335852,40.079235,49.411586,47.488400,58.636488,65.399162,39.877024,37.609373,48.730086,51.195256


In [13]:
leg_1_cols = list(filter(lambda c: "_1" in c,estimates.columns.values))
column_base_names = list(map(lambda c: c[:-2], leg_1_cols))
runner_estimates = pd.wide_to_long(estimates.reset_index(), stubnames=column_base_names, i ="team_id", j="leg", sep = "_")
runner_estimates = runner_estimates.sort_values(by=['team_id', 'leg'])
#runner_estimates = runner_estimates.drop(['team_base_name', 'estimated_log_means', 'estimated_log_stdevs'], axis=1)
runner_estimates.head(7)

team  dist                name  log_mean  \
team_id leg                                                              
1       1          Stora Tuna OK 1   6.0       Tilda Östberg    1.8857   
        2          Stora Tuna OK 1   5.7    Magdalena Olsson    1.9556   
        3          Stora Tuna OK 1   7.3         Julia Jakob    1.7824   
        4          Stora Tuna OK 1   7.9  Tove Alexandersson    1.8027   
2       1    Göteborg Majorna OK 1   6.0      Amanda Hansson    2.0053   
        2    Göteborg Majorna OK 1   5.7       Runa Fremstad    2.0081   
        3    Göteborg Majorna OK 1   7.3    Kristine Berglia    1.9581   

              log_std  fin_start95   fin_end95  fin_median    fin_mean  \
team_id leg                                                              
1       1    0.067659    34.575157   45.098622   39.566249   39.641248   
        2    0.088600    71.707841   89.106250   79.937558   80.031186   
        3    0.110076   111.591144  136.843093  123.544775  123.671780   
        4    0.067659   158.221538  186.331715  171.460249  171.720541   
2       1    0.077564    38.313828   51.842862   44.577523   44.718095   
        2    0.087900    77.769334   97.337735   87.242081   87.345612   
        3    0.067659   127.402399  151.443076  139.002581  139.161323   

             fin_sum_log_mean  fin_sum_log_std  mass_start  dark_during_leg  \
team_id leg                                                                   
1       1            3.677590         0.067546         0.0              0.0   
        2            4.380857         0.055840         0.0              0.0   
        3            4.816267         0.052236         0.0              0.0   
        4            5.144983         0.042059         0.0              0.0   
2       1            3.797368         0.077593         0.0              0.0   
        2            4.468211         0.057688         0.0              0.0   
        3            4.934660         0.044154         0.0              0.0   

             personal_start95  personal_end95  personal_median  
team_id leg                                                     
1       1           34.575157       45.098622        39.566249  
        2           33.768825       47.858028        40.252988  
        3           35.116472       53.721062        43.444549  
        4           42.004915       54.841958        47.911052  
2       1           38.313828       51.842862        44.577523  
        2           35.873987       50.313313        42.452216  
        3           45.309450       58.881223        51.688998

In [14]:
runner_estimates[runner_estimates['team'].str.contains("Reak")][["name", "fin_start95", "fin_median", "fin_end95", "mass_start", "dark_during_leg"]]

,,name,fin_start95,fin_median,fin_end95,mass_start,dark_during_leg
team_id,leg,,,,,,


In [15]:
# Convert minutes to date and times
start_timestamp = shared.start_timestamp[ve_or_ju][2019]

runner_estimates["fin_time_median"] = pd.to_datetime(runner_estimates["fin_median"] * 60, unit = "s", origin= start_timestamp)
runner_estimates["fin_time_start95"] = pd.to_datetime(runner_estimates["fin_start95"] * 60, unit = "s", origin= start_timestamp)
runner_estimates["fin_time_end95"] = pd.to_datetime(runner_estimates["fin_end95"] * 60, unit = "s", origin= start_timestamp)

runner_estimates["personal_estimate_interval"] = runner_estimates["personal_end95"] - runner_estimates["personal_start95"]
runner_estimates.head()

team  dist                name  log_mean  \
team_id leg                                                              
1       1          Stora Tuna OK 1   6.0       Tilda Östberg    1.8857   
        2          Stora Tuna OK 1   5.7    Magdalena Olsson    1.9556   
        3          Stora Tuna OK 1   7.3         Julia Jakob    1.7824   
        4          Stora Tuna OK 1   7.9  Tove Alexandersson    1.8027   
2       1    Göteborg Majorna OK 1   6.0      Amanda Hansson    2.0053   

              log_std  fin_start95   fin_end95  fin_median    fin_mean  \
team_id leg                                                              
1       1    0.067659    34.575157   45.098622   39.566249   39.641248   
        2    0.088600    71.707841   89.106250   79.937558   80.031186   
        3    0.110076   111.591144  136.843093  123.544775  123.671780   
        4    0.067659   158.221538  186.331715  171.460249  171.720541   
2       1    0.077564    38.313828   51.842862   44.577523   44.718095   

             fin_sum_log_mean  fin_sum_log_std  mass_start  dark_during_leg  \
team_id leg                                                                   
1       1            3.677590         0.067546         0.0              0.0   
        2            4.380857         0.055840         0.0              0.0   
        3            4.816267         0.052236         0.0              0.0   
        4            5.144983         0.042059         0.0              0.0   
2       1            3.797368         0.077593         0.0              0.0   

             personal_start95  personal_end95  personal_median  \
team_id leg                                                      
1       1           34.575157       45.098622        39.566249   
        2           33.768825       47.858028        40.252988   
        3           35.116472       53.721062        43.444549   
        4           42.004915       54.841958        47.911052   
2       1           38.313828       51.842862        44.577523   

                          fin_time_median              fin_time_start95  \
team_id leg                                                               
1       1   2019-06-15 14:39:33.974925756 2019-06-15 14:34:34.509437084   
        2   2019-06-15 15:19:56.253453015 2019-06-15 15:11:42.470487833   
        3   2019-06-15 16:03:32.686510324 2019-06-15 15:51:35.468627930   
        4   2019-06-15 16:51:27.614933014 2019-06-15 16:38:13.292270422   
2       1   2019-06-15 14:44:34.651402235 2019-06-15 14:38:18.829669237   

                           fin_time_end95  personal_estimate_interval  
team_id leg                                                            
1       1   2019-06-15 14:45:05.917302847                   10.523464  
        2   2019-06-15 15:29:06.374982119                   14.089203  
        3   2019-06-15 16:16:50.585586071                   18.604590  
        4   2019-06-15 17:06:19.902882099                   12.837043  
2       1   2019-06-15 14:51:50.571706295                   13.529034

In [16]:

for_print = runner_estimates.copy()
for_print = for_print.reset_index()
for_print = for_print.set_index('team_id')
for_print = for_print.round(2)
for_print["median"] = for_print["fin_time_median"].dt.strftime("%H:%M")
for_print["start95"] = for_print["fin_time_start95"].dt.strftime("%H:%M")
for_print["end95"] = for_print["fin_time_end95"].dt.strftime("%H:%M")

#for_print[for_print['team'].str.contains("Stadin")]
for_print = for_print[for_print['team'].str.contains("Jyry")]

for_print = for_print[[
 'leg',
 'name', 
 'personal_start95', 
 'personal_median',
 'personal_end95',
 'personal_estimate_interval']]

for_print.to_csv(f'data/for_print_{ve_or_ju}_2019.csv')
for_print

,leg,name,personal_start95,personal_median,personal_end95,personal_estimate_interval
team_id,,,,,,
341,1,Jola Maisala,41.69,54.17,69.85,28.16
341,2,Kaija Rantoja,45.26,58.40,75.75,30.49
341,3,Jenni Castren,57.33,75.39,98.80,41.47
341,4,Inka Lemmetyinen,55.06,78.40,111.06,56.00
528,1,Päivi Taponen,46.74,63.64,85.69,38.95
528,2,Mervi Luurila,47.61,67.60,97.18,49.57
528,3,Suvi Tiainen,72.15,103.57,149.62,77.47
528,4,Tiina Isoniemi,59.15,68.21,78.85,19.69
740,1,Mila Viksilä,55.74,79.46,113.53,57.78


In [17]:
for_web = runner_estimates.copy().reset_index()
for_web["fin_time_median"] = for_web["fin_time_median"].dt.tz_localize('EET').dt.tz_convert('UTC')
for_web["fin_time_start95"] = for_web["fin_time_start95"].dt.tz_localize('EET').dt.tz_convert('UTC')
for_web["ind_log_mean"] = for_web["log_mean"]
for_web["ind_log_std"] = for_web["log_std"]

for_web["fin_time_end95"] = for_web["fin_time_end95"].dt.tz_localize('EET').dt.tz_convert('UTC')

#for_web["fin_time_median"] = for_web["fin_time_median"].dt.strftime("%H:%M")
#for_web["fin_time_start95"] = for_web["fin_time_start95"].dt.strftime("%H:%M")
#for_web["fin_time_end95"] = for_web["fin_time_end95"].dt.strftime("%H:%M")
for_web = for_web[[
 'team_id',
 'leg',
 'team',
 'name',
 'ind_log_mean', 
 'ind_log_std',
 'fin_sum_log_mean', 
 'fin_sum_log_std',
  'fin_time_start95', 
 'fin_time_end95',
 'fin_time_median',
 'mass_start', 
 'dark_during_leg']]

for_web[for_web['team'].str.contains("Jyry")]
#for_web[for_web['team'].str.contains("Reak")]

,team_id,leg,team,name,ind_log_mean,ind_log_std,fin_sum_log_mean,fin_sum_log_std,fin_time_start95,fin_time_end95,fin_time_median,mass_start,dark_during_leg
1332,341,1,Rasti-Jyry 1,Jola Maisala,2.200700,0.130900,3.989870,0.132139,2019-06-15 11:41:41.431359768+00:00,2019-06-15 12:09:50.960183620+00:00,2019-06-15 11:54:10.121687651+00:00,0.0000,0.0
1333,341,2,Rasti-Jyry 1,Kaija Rantoja,2.328600,0.130600,4.727059,0.092939,2019-06-15 12:34:10.720040321+00:00,2019-06-15 13:15:34.519303083+00:00,2019-06-15 12:53:01.739399195+00:00,0.0000,0.0
1334,341,3,Rasti-Jyry 1,Jenni Castren,2.334200,0.141700,5.241677,0.079950,2019-06-15 13:41:44.784987688+00:00,2019-06-15 14:41:16.908539772+00:00,2019-06-15 14:09:07.512142897+00:00,0.0000,0.0
1335,341,4,Rasti-Jyry 1,Inka Lemmetyinen,2.293200,0.178309,5.592469,0.077073,2019-06-15 14:50:48.868538618+00:00,2019-06-15 16:11:57.019504786+00:00,2019-06-15 15:28:03.808332682+00:00,0.0000,0.0
2076,528,1,Rasti-Jyry 2,Päivi Taponen,2.361600,0.156778,4.152582,0.155226,2019-06-15 11:46:44.392123699+00:00,2019-06-15 12:25:41.649228573+00:00,2019-06-15 12:03:38.313879251+00:00,0.0000,0.0
2077,528,2,Rasti-Jyry 2,Mervi Luurila,2.473200,0.181082,4.884066,0.120077,2019-06-15 12:44:53.024252892+00:00,2019-06-15 13:47:20.863891125+00:00,2019-06-15 13:12:07.931344748+00:00,0.0000,0.0
2078,528,3,Rasti-Jyry 2,Suvi Tiainen,2.655400,0.187373,5.469207,0.107127,2019-06-15 14:12:11.276508331+00:00,2019-06-15 15:52:04.724397898+00:00,2019-06-15 14:57:13.366530895+00:00,0.0000,0.0
2079,528,4,Rasti-Jyry 2,Tiina Isoniemi,2.155400,0.073500,5.723261,0.084932,2019-06-15 15:19:13.050429821+00:00,2019-06-15 17:01:48.610629320+00:00,2019-06-15 16:05:29.561195612+00:00,0.1148,0.0
2916,740,1,Rasti-Jyry 4,Mila Viksilä,2.581289,0.182322,4.374106,0.180397,2019-06-15 11:55:44.669235706+00:00,2019-06-15 12:53:31.626432657+00:00,2019-06-15 12:19:27.661782980+00:00,0.0000,0.0
2917,740,2,Rasti-Jyry 4,Tiina Sidensnöre,2.275900,0.154400,4.911758,0.122951,2019-06-15 12:46:36.402002811+00:00,2019-06-15 13:53:03.078847885+00:00,2019-06-15 13:15:43.960308790+00:00,0.0000,0.0


In [18]:
for_web.to_json(f'web-lib/for_web_{ve_or_ju}_2019.json', orient="records", date_format="iso")

In [19]:
for_web.shape

(6728, 13)

In [20]:
np.exp([-0.075241, 0.0, 0.01, 0.5])

array([0.92751993, 1.        , 1.01005017, 1.64872127])